# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121716e+02     1.752293e+00
 * time: 0.09120702743530273
     1     1.089919e+01     1.107959e+00
 * time: 1.3596348762512207
     2    -1.151150e+01     1.337557e+00
 * time: 1.5110509395599365
     3    -3.411965e+01     8.686416e-01
 * time: 1.6509168148040771
     4    -4.759752e+01     5.919620e-01
 * time: 1.8023598194122314
     5    -5.671517e+01     2.110934e-01
 * time: 1.947774887084961
     6    -5.958329e+01     1.505753e-01
 * time: 2.062819004058838
     7    -6.080447e+01     6.228930e-02
 * time: 2.1700589656829834
     8    -6.132651e+01     4.541938e-02
 * time: 2.288674831390381
     9    -6.161629e+01     4.692077e-02
 * time: 2.39998197555542
    10    -6.184857e+01     2.721718e-02
 * time: 2.519171953201294
    11    -6.202446e+01     2.031502e-02
 * time: 2.6268680095672607
    12    -6.211132e+01     1.598944e-02
 * time: 2.744534969329834
    13    -6.216224e+01     1.204958e-02
 * time: 2.8610289096832275

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557655
    AtomicNonlocal      14.8522634
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485361 
    Xc                  -19.3336816

    total               -62.261666461282
